## EX2

Uma indústria dispõe de 3 recursos: **matéria-prima**, **mão de obra** e **tempo de maquinário**, em quantidades limitadas. Com esses recursos, a indústria pretende produzir dois produtos, **1** e **2**. O quadro abaixo dá a utilização de cada recurso em cada um dos produtos e a disponibilidade de cada recurso.

| Recurso          | Recurso gasto para fazer 1 unidade do produto 1 | Recurso gasto para fazer 1 unidade do produto 2 | Disponibilidade |
|------------------|-------------------------------------------------|-------------------------------------------------|-----------------|
| **Matéria-Prima**    | 1                                               | 2                                               | 14              |
| **Mão de Obra**      | 1                                               | 1                                               | 9               |
| **Tempo de Maquinário** | 7                                             | 4                                               | 56              |

A indústria sabe que cada unidade produzida do **produto 1** dá um lucro de **R$ 5,00** e do **produto 2** um lucro de **R$ 6,00**.


###  Modelo Matemático

#### Variáveis 



p1 = quantidade do produto 1 a ser produzida 

p2 = quantidade do produto 2 a ser produzida

#### Função Objetivo

$Max \quad Z = 5*p1+6*p2$

#### Restrições

- Limite quantidade de materia prima

$p1*1 + 2*p2 = 14$

- Limite Mão de obra

$p1*1 + 1*p2 = 9$

- Limite tempo maquinário

$p1*7 + 4*p2 = 56$




In [10]:
import pyomo.environ as pyo
import time

In [11]:
# Criação do modelo
model = pyo.ConcreteModel("Produtos")

In [12]:
# Tipos produto:
model.P = pyo.Set(initialize=["1", "2"])

In [13]:
# ------------------------------
# Parâmetros
# ------------------------------

# lucro produto material
luc = {"1":5, "2":6}
model.lucro = pyo.Param(model.P,initialize=luc)

# Custos material
material = {"1":1, "2":2}
model.mat = pyo.Param(model.P,initialize=material)

# Mão de obra
mao_obra = {"1":1, "2":1}
model.mao_de_obra = pyo.Param(model.P,initialize=mao_obra)

# Tempo maquinario
maq = {"1":7, "2":4}
model.temp_maq = pyo.Param(model.P,initialize=maq)

# Material disponível
model.e_material = pyo.Param(initialize=14)

#Mão de obra disponivel
model.e_mao_obra = pyo.Param(initialize=9)

#Tempo Maquinario disponivel
model.e_temp_maq = pyo.Param(initialize=56)

In [14]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# quantidade da ração tobi e max
model.x = pyo.Var(model.P,domain=pyo.NonNegativeIntegers)

In [15]:
# ------------------------------
# Função Objetivo
# ------------------------------
def objective_rule(model):
    # Lucro por produto
    valor_pacote = sum(
        model.x[i] * model.lucro[i]
        for i in model.P
    )

    return valor_pacote


model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

In [16]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite Materia
def meteria(model):
    return sum(model.x[i] * model.mat[i] for i in model.P) <= model.e_material

model.re_materia= pyo.Constraint(rule=meteria)

# 2. Limite Maõ de Obra
def mao_obra(model):
    return sum(model.x[i] * model.mao_de_obra[i] for i in model.P) <= model.e_mao_obra

model.re_mao_obra = pyo.Constraint(rule=mao_obra)

# 3. Tempo maquinario
def maquinario(model):
    return sum(model.x[i] * model.temp_maq[i] for i in model.P) <= model.e_temp_maq

model.re_maq = pyo.Constraint(rule=maquinario)


In [17]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex2.lp", io_options={"symbolic_solver_labels": True})

# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.obj)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



MIP  has 3 rows; 2 cols; 6 nonzeros; 2 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 7e+00]
  Cost   [5e+00, 6e+00]
  Bound  [0e+00, 0e+00]
  RHS    [9e+00, 6e+01]
Presolving model
3 rows, 2 cols, 6 nonzeros  0s
3 rows, 2 cols, 6 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   3 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   6 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 z       0       0         0   0.00% 

In [18]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto 1 : 4.00
  Quantidade do produto 2 : 5.00
